In [10]:
import datetime
from datetime import datetime
import pandas as pd
import time
import requests
import os
import numpy as np
import xlsxwriter
from abc import ABCMeta, abstractmethod
from Queue import Queue
from collections import deque
import getpass
import pandas.io.formats.excel
from matplotlib import rc
import matplotlib.pyplot as plt

### Simulation runner

In [76]:

class SimulationEnv(object):
    
    loggables = ['positions']
    
    def __init__(self, wealth, assets, 
                 path_to_data, start_date, end_date,
                 agent_type, expert_type, reinvest, max_assets=100):

        self.init_wealth = wealth
        self.wealth = wealth
        self.assets = assets
        self.agent_type = agent_type
        self.expert_type = expert_type
        self.path_to_data = path_to_data
        self.reinvest = reinvest
        self.max_assets = max_assets
        self.start_date = start_date
        self.end_date = end_date
        
        return
    
    ## To be called before running a simulation - initialize experts, agents, and initial position, etc.
    def setup_params(self, agent_args={}, expert_args={}):
    
        stdevs = {}
        for f in os.listdir(self.path_to_data):
            df = pd.read_csv(self.path_to_data+f)
            df = df[df["date"] < self.end_date]
            df["return"] = (df["adj_close"] - df["adj_close"].shift(1))/df["adj_close"].shift(1)
            stdevs[f] = df["return"].std()
            self.assets = [f.split(".")[0].upper() for f in sorted(stdevs, key=stdevs.get, reverse=True)[:self.max_assets]]
        
        print self.assets
        
        self.experts = [self.expert_type(a, self.path_to_data, self.start_date, self.end_date, **expert_args) for a in self.assets]
        self.agent = self.agent_type(self.experts, **agent_args)
        self.positions = np.array([weight * self.wealth for weight in self.agent.weights])
        
        return
        
    ## Run simulation
    def run(self, log=False, logpath=os.getcwd()):
        
        ## Start period counter and log
        self.period = 1
        self.finallog = []
        self.log = log
        self.logpath = logpath
        
        ## Warmup period: 
        ## i.e. for strategies involving moving average indicators, wait until we have enough data to calculate MA
        for expert in self.agent.experts:
            expert.warmup()
        
        ## Simulation: Go until data iterators reach the end
        print "Initial weights:"
        print np.array([weight for weight in self.agent.weights])
        
        while True:
            try:
                print "PERIOD {}".format(self.period)
                print "dates:"
                dates = [e.current_date for e in self.agent.experts]
                print list(set(dates))
                ## NEED TO MAKE THIS TRUE
                ## assert len(set(dates)) == 1
              


                print "---------------------"
                ## Log this period
                if log:
                    self.logperiod()
                    
                ## Update experts with last period's rewards
                for i, expert in enumerate(self.agent.experts):
                    expert.update()

                ## Update agent accordingly (i.e. for Hedge, update weights according to each expert's reward in the last period)
                self.agent.update()
                           
                ## Rewards accrue to agent
                self.positions = self.positions * (1 + self.agent.returns)
                self.wealth = np.sum(self.positions)
                
                ## Rebalance according to new, updated weights
                ## TODO: only allow non-fractional shares to be purchased (?)
                if np.sum(self.agent.weights) == 1:
                    self.positions = np.array([weight * self.wealth for weight in self.agent.weights])
                
#                 print "weights:"
#                 print np.array([weight for weight in self.agent.weights])

#                 print "rewards:"
#                 print np.array([r for r in self.agent.rewards])
#                 print "returns:"
#                 print np.array([r for r in self.agent.returns])
                
#                 print "positions:"
#                 print np.array([weight * self.wealth for weight in self.agent.weights])
#                 print "positions invested:"
#                 positions_invested = np.array([e.pick for e in self.agent.experts])*self.positions
#                 print positions_invested
#                 if self.reinvest and np.any(positions_invested):
#                     uninvested_wealth = self.wealth - np.sum(positions_invested)
#                     print "Need to reallocate {} cash amongst experts that are investing".format(uninvested_wealth)
#                     print "Total wealth:"
#                     print self.wealth
#                     print "Reinvestments:"
#                     print uninvested_wealth * positions_invested/np.sum(positions_invested)
#                     print "Final position"
#                     self.positions = uninvested_wealth * positions_invested/np.sum(positions_invested) + positions_invested
#                     print self.positions
#                     new_wealth = np.sum(self.positions)
#                     assert(new_wealth-self.wealth <= 0.00001)
                
#                 print "---------------------"
#                 print "\n\n"
                
                
                ## Advance period
                self.period += 1
                
            except StopIteration:
                break
        
        ## Write the log file
        if log:
            self.savelog()

    def logperiod(self):
        row = [self.period] + [self.wealth]
        nrow = []
        for loggable in (self.loggables):
            if getattr(self,loggable) is None:
                nrow += [None] * len(self.experts)
            else:
                nrow += getattr(self,loggable).tolist()
        arow = []
        for loggable in (self.agent_type.loggables):
            if getattr(self.agent,loggable) is None:
                arow += [None] * len(self.experts)
            else:
                arow += getattr(self.agent,loggable).tolist()
        erow = []
        for loggable in (self.expert_type.loggables):
            if [getattr(e,loggable) for e in self.experts] is None:
                erow += [None] * len(self.experts)
            else:
                erow += [getattr(e,loggable) for e in self.experts]
        row += nrow + arow + erow
        self.finallog.append(row)
    
    def savelog(self):
        # Set up log file structure
        runtime = datetime.datetime.now()
        runuser = getpass.getuser()
        logname = runuser + "_" + runtime.strftime('%Y-%m-%d_%H-%M-%S')
        if not os.path.exists(os.path.join(self.logpath, logname)):
            os.makedirs(os.path.join(self.logpath, logname))
            os.makedirs(os.path.join(self.logpath, logname, 'plots'))
        
        # Create dataframe from log
        col = ['period', 'wealth'] + \
            [x+'.'+y for x in self.loggables for y in self.assets] + \
            [x+'.'+y for x in self.agent_type.loggables for y in self.assets] + \
            [x+'.'+y for x in self.expert_type.loggables for y in self.assets]
        df = pd.DataFrame(self.finallog, columns=col)
        df.set_index('period', inplace=True)
        
        # Write xlsx of log data
        writer = pd.ExcelWriter(os.path.join(self.logpath, logname, logname+'.xlsx'), engine='xlsxwriter')
        pd.io.formats.excel.header_style = None
        df.to_excel(writer,'run_log')
        
        workbook  = writer.book
        worksheet = writer.sheets['run_log']
        
        header_format = workbook.add_format({'bold': True,'text_wrap': True})
        worksheet.set_row(0, None, header_format)
        writer.save()
        
        # Set up matplotlib. Loop through loggables and save a plot with and without legend for each.
        rc('font', family = 'serif', serif = 'cmr10')
        plots = ['wealth'] + self.loggables + self.agent_type.loggables + self.expert_type.loggables
        
        for p in plots:
            for legend in [True, False]:
                plotdf = df.filter(regex='period|'+p)
                plotlab = [l.split(p+'.',1)[1] if l != 'wealth' else l.title() for l in list(plotdf.columns.values)]
                plt.plot(plotdf)
                plt.ylabel(p.title())
                plt.xlabel('Round')
                plt.ticklabel_format(useOffset=False)
                if legend:
                    plt.legend(plotlab, loc=9, bbox_to_anchor=(0.5, -0.15), ncol=4)
                    plt.savefig(os.path.join(self.logpath, logname, 'plots', p+'_legend.png'), bbox_inches='tight', dpi=300)
                else:
                    plt.savefig(os.path.join(self.logpath, logname, 'plots', p+'.png'), bbox_inches='tight', dpi=300)
                plt.close()



### Base classes

Base classes for agents and experts to be implemented by us

In [12]:

class Agent(object):
    __metaclass__ =ABCMeta
    
    @abstractmethod
    def __init__(self):
        pass
    
    @abstractmethod
    def update(self):
        pass
    
    @abstractmethod
    def act(self):
        pass
    
class Expert(object):
    __metaclass__=ABCMeta
    
    ## Required parameters for initializing experts
    @abstractmethod
    def __init__(self, name, path_to_data, start_date, end_date):
        self.reward = 0.
        self.pick = False ## Need to activate experts in child classes
        self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
        self.current_date = datetime.strptime("1000-01-01", "%Y-%m-%d")
        self.start_date = datetime.strptime(start_date, "%Y-%m-%d")
        self.end_date = datetime.strptime(end_date, "%Y-%m-%d")
        while self.current_date < self.start_date:
            self.last_row = self.data.get_chunk(1)
            self.last_price = float(self.last_row["adj_close"])
            self.current_date = datetime.strptime(self.last_row["date"].item(), "%Y-%m-%d")
        
    @abstractmethod
    def update(self):
        pass
    
class Context(object):
    __metaclass__=ABCMeta
    
    @abstractmethod
    def __init__(self, context_data):
        pass
    
    @abstractmethod
    def observe(self):
        pass

### Agents

Implementations of different online portfolio selection algorithms
* Exponential Gradient
* Exponential Gradient (recent history only)

In [13]:
## EG. (http://www.cis.upenn.edu/~mkearns/finread/helmbold98line.pdf)
class EG(Agent):
    
    loggables = ['returns','rewards','weights']
    
    ## Initialize with a set of experts
    def __init__(self, experts, eta):
        self.eta = eta
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        self.returns = None
        return
    
    ## Update the agent's rewards and its weights for each expert
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = np.asarray([e.returns for e in self.experts])
        multipliers = np.exp(self.eta * self.rewards/np.sum(self.weights * self.rewards))
        self.weights = (self.weights * multipliers)/ np.sum(self.weights * multipliers)
        self.weights = np.nan_to_num(self.weights)

        return
    
    def act(self):
        return self.weights


class EGRecent(Agent):
    
    loggables = ['returns','rewards','weights']
    
    ## Initialize with a set of experts
    def __init__(self, experts, eta, n):
        self.eta = eta
        self.experts = experts
        #self.weights = Queue(maxsize=n)
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = deque(maxlen=n)
        self.returns = None
        self.t = 0
        self.n = n
        #self.rewards = None
        #self.returns = None
        return
    
    ## Update the agent's rewards and its weights for each expert
    def update(self):
           
        self.rewards.append(np.asarray([e.reward for e in self.experts]))
        self.returns = np.asarray([e.returns for e in self.experts])
        
        self.weights = np.ones(len(self.experts))/len(self.experts)
        #print self.t
        #print len(self.rewards)
        for rewards in self.rewards:
            rewards = np.asarray(rewards)
            multipliers = np.exp(self.eta * rewards/np.sum(self.weights * rewards))
            self.weights = (self.weights * multipliers)/ np.sum(self.weights * multipliers)
            self.weights = np.nan_to_num(self.weights)
        self.t += 1
        #print self.t
        
        return
    
    def act(self):
        return self.weights
    
      
        

More agents to serve as naive/benchmark portfolio allocation algorithms:
* Equal-weighted buy and hold
* Market-weighted buy and hold
* Constantly rebalanced portfolio


In [14]:
## Some other examples of agents to use as benchmarks 

class ConstantRebalancer(Agent):
    
    loggables = ['rewards','weights']
    
    def __init__(self, experts):
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        return
    
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = self.rewards - 1.
        return
    
    def act(self):
        return self.weights
    
## TODO
class WeightedBuyHold(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return
    

## to be used with "Dummy" Expert for each stock
class NaiveBuyHold(Agent):
    
    def __init__(self, experts):
        self.experts = experts
        self.weights = np.ones(len(self.experts))/len(self.experts)
        self.rewards = None
        return
    
    def update(self):
        self.rewards = np.asarray([e.reward for e in self.experts])
        self.returns = self.rewards - 1.
        self.weights = np.multiply(self.weights, self.rewards)
        self.weights = np.divide(self.weights, np.sum(self.weights))
        assert((np.sum(self.weights) - 1.) <= 0.00001)
        return
    
    def act(self):
        return self.weights

### Experts

* Dummy
* Mean Reversion
* Momentum

In [26]:

## Dummy expert that always pick the same asset
class Dummy(Expert):
    
    loggables = ['reward']
    
    ## Expert has a reward associated with its pick
    def __init__(self, name, path_to_data, start_date, end_date):
        super(Dummy, self).__init__(name, path_to_data, start_date, end_date)
        self.pick = True
        return
    
    ## Expert updates its reward 
    def update(self):
        self.last_row = self.data.get_chunk(1)
        self.current_date = datetime.strptime(self.last_row["date"].item(), "%Y-%m-%d")
        
        if self.current_date > self.end_date:
            raise StopIteration
            
        current_price = float(self.last_row["adj_close"])
        self.reward = current_price/self.last_price
        self.returns = self.reward - 1.
        self.last_price = current_price
        return
    
    def warmup(self):
        pass
    

class MeanReversion(Expert):
    
    loggables = ['avg','std']
    
    def __init__(self, name, path_to_data, start_date, end_date, window_size, threshold):
        super(MeanReversion, self).__init__(name, path_to_data, start_date, end_date)
        self.window_size = window_size
        self.avg = 0.0
        self.std = 0.0
        self.threshold = threshold
        self.last_n_prices = Queue(maxsize=10)
        self.returns = 0.

        return
    
    def warmup(self):
        n = 1
        while n <= self.window_size:
            self.last_n_prices.put(self.last_price)
            
            self.last_row = self.data.get_chunk(1)
            self.current_date = datetime.strptime(self.last_row["date"].item(), "%Y-%m-%d")

            self.last_price = float(self.last_row["adj_close"])
            n += 1
        return
        
    def update(self):
        _ = self.last_n_prices.get()
        
        self.last_n_prices.put(self.last_price)
        self.avg = np.mean(list(self.last_n_prices.queue))
        self.std = np.std(list(self.last_n_prices.queue))
        
        self.last_row = self.data.get_chunk(1)
        self.current_date = datetime.strptime(self.last_row["date"].item(), "%Y-%m-%d")
        
        if self.current_date > self.end_date:
            raise StopIteration
            
        current_price = float(self.last_row["adj_close"])
        
        ## If self.pick is True, we bought the stock and our reward is whatever the return was in the last period
        if self.pick:
            self.reward = current_price/self.last_price
            self.returns = self.reward - 1.
        else:
            self.reward = self.last_price/current_price
            self.returns = 0
            
        self.last_price = current_price

        if self.last_price <= self.avg - self.threshold * self.std:
            self.pick = True
        else:
            self.pick = False

        return

    
# class Momentum(Expert):
    
#     loggables = ['avg','std']
    
#     def __init__(self, name, path_to_data, window_size, threshold):
#         self.reward = 0.
#         self.pick = False
#         self.data = pd.read_csv(path_to_data + name + ".csv", iterator=True, chunksize=1)
#         self.last_price = float(self.data.get_chunk(1)["adj_close"])
#         self.window_size = window_size
#         self.avg = 0.0
#         self.std = 0.0
#         self.threshold = threshold
#         self.last_n_prices = Queue(maxsize=10)
#         self.returns = 0.
#         return
    
#     def warmup(self):
#         n = 1
#         while n <= self.window_size:
#             self.last_n_prices.put(self.last_price)
#             self.last_price = float(self.data.get_chunk(1)["adj_close"])
#             n += 1
#         return
        
#     def update(self):
#         _ = self.last_n_prices.get()
            
#         self.last_n_prices.put(self.last_price)
#         self.avg = np.mean(list(self.last_n_prices.queue))
#         self.std = np.std(list(self.last_n_prices.queue))
        
#         current_price = float(self.data.get_chunk(1)["adj_close"])

#         ## If self.pick is True, we bought the stock and our reward is whatever the return was in the last period
#         if self.pick:
#             self.reward = current_price/self.last_price
#             self.returns = self.reward - 1.
#         else:
#             self.reward = -current_price/self.last_price
#             self.returns = 0
#         self.last_price = current_price

#         if self.last_price >= self.avg - self.threshold * self.std:
#             self.pick = True
#         else:
#             self.pick = False

#         return
  

### Contexts

In [85]:
def GridSearch(Agent, Expert, reinvest=False, full_data=False, agent_args=[], expert_args=[]):
    ## Run simulation 
    start = time.time()
    if full_data:
        data = "data/djia_20000101_20171101/"
        start_date = "2000-01-01"
        year = 17.
    else:
        data = "data/djia_20150101_20171101/"
        start_date = "2015-01-01"
        year = 2.
    best_wealth = 0
    best_params = {}
    for agent_arguments in agent_args:
        for expert_arguments in expert_args:
            
            s = SimulationEnv(
                initial_wealth, 
                stocks, 
                data, 
                start_date,
                "2017-11-01", 
                Agent, 
                Expert, 
                reinvest
            )
            s.setup_params(
                agent_args=agent_arguments,
                expert_args=expert_arguments
            )
            s.run()

            years = year + 11./12
            ar = ((s.wealth)/initial_wealth)**(1/years) - 1
            end = time.time()

            if s.wealth > best_wealth:
                best_wealth = s.wealth
                best_params = {
                    "agent_args": agent_arguments,
                    "expert_args": expert_arguments,
                    "Initial wealth": initial_wealth,
                    "Final wealth": s.wealth,
                    "Annualized return": ar,
                    "Time": int(end-start),
                }
    print("Best params:", best_params)
    
#GridSearch(ConstantRebalancer, Dummy, reinvest=True, full_data=False, agent_args=[{}], expert_args=[{}])
#GridSearch(EG, Dummy, reinvest=True, full_data=False, agent_args=[{"eta":-0.01},{"eta":-0.005}], expert_args=[{}])


In [8]:
  
class VolContext(Context):
    
    def __init__(self, data_file, threshold):
        self.data = pd.read_csv(data_file, iterator=True, chunksize=1)
        self.threshold = threshold
        self.contexts = ["HighVol", "LowVol"]
        return
    
    # Returns a string giving the name of the context
    def observe(self):
        if float(self.data.get_chunk(1)["adj_close"]) > self.threshold:
            return self.contexts[0]
        else:
            return self.contexts[1]

In [17]:
stocks = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GE',
 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 
'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'VZ', 'WMT', 'XOM']
stocks = [stock.lower() for stock in stocks]

In [18]:
initial_wealth = 1.

In [82]:
## Run simulation where constantly rebalance investments
start = time.time()
s = SimulationEnv(1., stocks, "data/djia_20150101_20171101/", "2016-01-01", "2016-12-31", EG, Dummy, False, max_assets=2)
s.setup_params(agent_args={"eta": 0.01})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

In [79]:
# Run simulation with sophisticated experts and fully invested wealth
start = time.time()
s = SimulationEnv(1., stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, MeanReversion, reinvest=True, max_assets=15)
s.setup_params(
    agent_args={"eta": 0.01}, 
    expert_args={"window_size": 10, "threshold": .5})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

Initial wealth: $1.0
Final wealth: $3.99557828344
Annualized return: 0.0803800058262
Time: 111s


In [83]:
## Run simulation where constantly rebalance investments
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", NaiveBuyHold, Dummy, True)
s.setup_params(agent_args={})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

In [84]:
## Run simulation where constantly rebalance investments
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", ConstantRebalancer, Dummy, True)
s.setup_params(agent_args={})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

In [ ]:
## Run simulation where constantly rebalance investments
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", ConstantRebalancer, Dummy, True)
s.setup_params(expert_args={"eta": -0.01})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print("Initial wealth: ${}".format(initial_wealth))
print("Final wealth: ${}".format(s.wealth))
print("Annualized return: {}".format(ar))
print("Time: {}s".format(int(end-start)))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock
start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01})
s.run(log=True, logpath="logs")

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock
start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EGRecent, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01, "n": 100})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run a simulation with Hedge, each expert is a proxy for buying a given stock

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, Dummy, True)
s.setup_params(agent_args = {"eta": -0.01, "n": 750})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run simulation where we just buy and hold all stocks in equal amounts
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", NaiveBuyHold, Dummy, True)
s.setup_params(expert_args={})
s.run()

years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))



In [ ]:
## Run simulation where we just buy and hold all stocks in equal amounts
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", NaiveBuyHold, Dummy, True)
s.setup_params(expert_args={})
s.run()

years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1
end = time.time()

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
## Run simulation where we each expert only recommends to buy when the price crosses 0.5 standard deviations below the 10-day moving average. 
## Otherwise doesn't do anything
## Sell position when price gets above 0.5 standard deviations below the 10-day MA

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, MeanReversion, True)

s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 2. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, MeanReversion, True)

s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:

start = time.time()

s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, MeanReversion, True)

s.setup_params(
    agent_args={"eta": -0.01, "n": 1250},
    expert_args={"window_size": 10, "threshold": .5}
)

s.run()
end = time.time()
years = 17. + 11./12
ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20150101_20171101/", "2015-01-01", "2017-11-01", EG, Momentum, True)

years = 2. + 11./12
s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EG, Momentum, True)

years = 17. + 11./12
s.setup_params(
    agent_args={"eta": -0.01},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))

In [ ]:
start = time.time()
s = SimulationEnv(initial_wealth, stocks, "data/djia_20000101_20171101/", "2000-01-01", "2017-11-01", EGRecent, Momentum, True)

years = 17. + 11./12
s.setup_params(
    agent_args={"eta": -0.001, "n": 500},
    expert_args={"window_size": 10, "threshold": 2.5}
)

s.run()
end = time.time()

ar = ((s.wealth)/initial_wealth)**(1/years) - 1

print "Initial wealth: ${}".format(initial_wealth)
print "Final wealth: ${}".format(s.wealth)
print "Annualized return: {}".format(ar)
print "Time: {}s".format(int(end-start))